In [1]:
#!/usr/bin/env python3
import wmfdata as wmf
import pandas as pd
import matplotlib

matplotlib.style.use('ggplot')

spark = wmf.spark.get_custom_session(
    master="yarn",
    spark_config={
        "spark.driver.memory": "16g",
        "spark.dynamicAllocation.maxExecutors": 128,
        "spark.executor.memory": "16g",
        "spark.executor.cores": 4,
        "spark.sql.shuffle.partitions": 512
    }
)

You are using wmfdata v1.3.2, but v1.3.3 is available.

To update, run `pip install --upgrade git+https://github.com/wikimedia/wmfdata-python.git@release --ignore-installed`.

To see the changes, refer to https://github.com/wikimedia/wmfdata-python/blob/release/CHANGELOG.md
PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.


PYSPARK_PYTHON=/usr/lib/anaconda-wmf/bin/python3


SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/usr/lib/spark2/jars/slf4j-log4j12-1.7.16.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/usr/lib/hadoop/lib/slf4j-log4j12-1.7.25.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.slf4j.impl.Log4jLoggerFactory]
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/08/23 21:12:59 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).
22/08/23 21:13:01 WARN Utils: Service 'sparkDriver' could not bind on port 12000. Attempting port 12001.
22/08/23 21:13:01 WARN Utils: Service 'sparkDriver' could not bind on port 12001. Attempting port 12002.
22/08/23 21:13:01 

In [3]:
LANGUAGE_EDITIONS = "('enwiki','eswiki','jawiki','arwiki','dewiki','ptwiki','viwiki','frwiki','cawiki','ruwiki','idwiki','bnwiki','hiwiki','cswiki')"

query = """
SELECT
    year(MIN(date(substr(event_timestamp,1,10)))) as year, 
    month(MIN(date(substr(event_timestamp,1,10)))) as month,
    wiki_db
FROM wmf.mediawiki_history   
WHERE snapshot = '2022-07'
AND wiki_db in {0}
AND array_contains(event_user_groups,'bot') = FALSE
AND revision_is_identity_reverted = FALSE
AND revision_text_bytes_diff > 0
GROUP BY
    wiki_db
    
""".format(
    LANGUAGE_EDITIONS
)

df = wmf.spark.run(query)
df

PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.


,year,month,wiki_db
0,2002,11,viwiki
1,2002,2,hiwiki
2,2002,4,jawiki
3,2001,6,ptwiki
4,2003,7,arwiki
5,2003,1,idwiki
6,2001,5,eswiki
7,2001,10,frwiki
8,2001,4,dewiki
9,2001,3,cawiki


In [5]:
FILEPATH = '/home/jmads/datasets/momentum/language_edition_first_edit_timestamps_8-23-22.csv'

df.to_csv(FILEPATH,index=False)